In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1]:
MUSIC_CSV_PATH = '/csvfiles/musiccaps-public.csv'
TRAIN_DATA = 'data/train_data.pkl'
TEST_DATA = 'data/test_data.pkl'
VAL_DATA = 'data/val_data.pkl'
URL_BASE = 'https://www.youtube.com/watch?v='
MELS_DIR = '/MELS_DIR'
AUDIO_DATA9 = 'audio_data9.pkl'

In [13]:
# TRAIN_DATA = 'new_data/train_data.pkl'
# TEST_DATA = 'new_data/test_data.pkl'
# VAL_DATA = 'new_data/val_data.pkl'

In [32]:
import torch
import librosa
import numpy as np
import pandas as pd
import torch.nn as nn
import soundfile as sf
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [3]:
train_audio_df = pd.read_pickle(TRAIN_DATA)

In [5]:
val_audio_df = pd.read_pickle(VAL_DATA)

In [ ]:
train_audio_df

In [6]:
for row in val_audio_df.iterrows():
    print(row[1]['mel_spec'].shape)

torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 431])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Size([128, 431])
torch.Size([128, 431])
torch.Size([128, 1])
torch.Size([128, 1])
torch.Si

### Padding function

In [7]:
def pad_mel_spec(mel_spec, target_shape=(128, 431)):
    padded = torch.zeros(target_shape)
    _, orig_width = mel_spec.shape
    if orig_width < target_shape[1]:
        padded[:, :orig_width] = mel_spec
    else:
        padded = mel_spec[:, :target_shape[1]]
    return padded

### Normalizing mel spectograms

In [8]:
train_audio_df['padded_mel_spec'] = train_audio_df['mel_spec'].apply(lambda x: pad_mel_spec(x))

train_audio_df['normalized_mel_spec'] = train_audio_df['padded_mel_spec'].apply(lambda x: (x - x.min())/(x.max() - x.min()))
 
mel_specs = torch.stack(train_audio_df['normalized_mel_spec'].tolist())

In [8]:
def preprocessing(df):
    df['padded_mel_spec'] = df['mel_spec'].apply(lambda x: pad_mel_spec(x))
    df['normalized_mel_spec'] = df['padded_mel_spec'].apply(lambda x: (x - x.min())/(x.max() - x.min()))
    mel_specs = torch.stack(df['normalized_mel_spec'].tolist())
    return mel_specs

In [9]:
mel_specs = preprocessing(train_audio_df)

In [13]:
val_spectos = preprocessing(val_audio_df)

In [11]:
mel_specs

tensor([[[0.5228, 0.4700, 0.4498,  ..., 0.5334, 0.5809, 0.5759],
         [0.5017, 0.4436, 0.4273,  ..., 0.4969, 0.5210, 0.5317],
         [0.5058, 0.4808, 0.4321,  ..., 0.3792, 0.3478, 0.4611],
         ...,
         [0.1467, 0.1820, 0.1790,  ..., 0.2137, 0.1656, 0.1671],
         [0.0000, 0.0014, 0.0436,  ..., 0.2256, 0.2012, 0.2013],
         [0.0000, 0.0000, 0.0000,  ..., 0.1955, 0.1961, 0.1996]],

        [[0.5892, 0.6143, 0.6116,  ..., 0.3888, 0.3955, 0.3270],
         [0.6252, 0.6526, 0.6681,  ..., 0.3610, 0.4184, 0.4135],
         [0.7017, 0.7585, 0.7629,  ..., 0.5516, 0.5598, 0.5396],
         ...,
         [0.1917, 0.1395, 0.0772,  ..., 0.0000, 0.0000, 0.0000],
         [0.2126, 0.1619, 0.0972,  ..., 0.0000, 0.0000, 0.0000],
         [0.2067, 0.1494, 0.0671,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.

### Dataloader

In [14]:
dataset = TensorDataset(mel_specs)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [15]:
valset = TensorDataset(val_spectos)
val_loader = DataLoader(valset, batch_size=64, shuffle=True)

### VAE defenition

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc31 = nn.Linear(128, latent_dim)  
        self.fc32 = nn.Linear(128, latent_dim) 

    def forward(self, x):
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, output_dim)

    def forward(self, z):
        h = torch.relu(self.fc1(z))
        h = torch.relu(self.fc2(h))
        return torch.sigmoid(self.fc3(h))

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar


### Loss function

In [17]:
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

### Initializing VAE model

In [11]:
input_dim = 431
latent_dim = 64
learning_rate = 0.001
num_epochs = 200

model = VAE(input_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Extracting latent vector

In [57]:
latent_vectors = []

model.eval()  
with torch.no_grad():
    for batch in data_loader:
        batch = batch[0]  
        batch = batch.view(-1, 431)  
        mu, logvar = model.encoder(batch)
        z = model.reparameterize(mu, logvar)
        latent_vectors.append(z)

latent_vectors = torch.cat(latent_vectors, dim=0)


In [58]:
latent_vectors

tensor([[ 0.0615,  0.7482,  0.2182,  ..., -0.3423, -1.2837, -0.5314],
        [-0.7227, -0.7161, -0.2857,  ...,  0.9881, -1.3810, -1.3239],
        [-0.9438, -0.7801,  0.3344,  ...,  1.4332, -1.3382, -0.8896],
        ...,
        [ 0.6796,  0.3389,  0.1825,  ...,  0.1115,  0.8167, -0.3806],
        [ 0.1171, -0.1901, -0.7484,  ..., -0.3234, -0.4864,  0.0076],
        [-0.3680, -0.1354,  0.1007,  ...,  0.1832, -0.7327,  1.7437]])

In [59]:
torch.save(model.state_dict(), "vae.pth")

## Under devlopment

### Using Latent space to generate noise and decode to mel spectogram

In [18]:
import torch
import torch.nn as nn

class DDIM(nn.Module):
    def __init__(self, latent_dim, timesteps):
        super(DDIM, self).__init__()
        self.latent_dim = latent_dim
        self.timesteps = timesteps
        self.noise_schedule = self._build_noise_schedule()
        self.noise_predictor = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )

    def _build_noise_schedule(self):
        return torch.linspace(1e-4, 0.02, self.timesteps)

    def forward_diffusion(self, z):
        noisy_z = []
        current_z = z
        for t in range(self.timesteps):
            noise = torch.randn_like(current_z)
            current_z = current_z + noise * self.noise_schedule[t]
            noisy_z.append(current_z)
        return noisy_z

    def reverse_diffusion(self, noisy_z):
        estimated_z = noisy_z[-1]
        for t in reversed(range(self.timesteps)):
            noise = torch.randn_like(estimated_z)
            estimated_z = estimated_z - noise * self.noise_schedule[t]
        return estimated_z

    def forward(self, z):
        noisy_z = self.forward_diffusion(z)
        estimated_z = self.reverse_diffusion(noisy_z)
        return estimated_z


### Chaining VAE & DDIM

In [19]:
input_dim = 128 * 431
latent_dim = 64
learning_rate = 0.001
num_epochs = 200


In [20]:

vae = VAE(input_dim, latent_dim)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)

timesteps = 100
ddim = DDIM(latent_dim, timesteps)
ddim_optimizer = optim.Adam(ddim.parameters(), learning_rate)


In [30]:
for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
# VAE    
    for batch in data_loader:
        batch = batch[0]
        optimizer.zero_grad()
        batch = batch.view(-1, input_dim)
        recon_batch, mu, logvar = vae(batch)
        loss = loss_function(recon_batch, batch, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, VAE_Loss: {train_loss / len(data_loader):.4}")

# DDIM
    ddim_loss = 0
    for batch in data_loader:
        batch = batch[0]
        batch = batch.view(-1, input_dim)
        mu, logvar = vae.encoder(batch)
        z = vae.reparameterize(mu, logvar)
        estimated_z = ddim(z)
        loss = F.mse_loss(estimated_z, z)
        ddim_optimizer.zero_grad()
        loss.backward()
        ddim_optimizer.step()
        ddim_loss += loss.item()
        
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, DDIM Loss: {ddim_loss / len(data_loader):.4}")


Epoch 0, VAE_Loss: 3.852e+06
Epoch 0, DDIM Loss: 0.02678
Epoch 10, VAE_Loss: 526.7
Epoch 10, DDIM Loss: 0.02718
Epoch 20, VAE_Loss: 1.893
Epoch 20, DDIM Loss: 0.02688
Epoch 30, VAE_Loss: 1.821
Epoch 30, DDIM Loss: 0.02665
Epoch 40, VAE_Loss: 16.87
Epoch 40, DDIM Loss: 0.02672
Epoch 50, VAE_Loss: 1.755
Epoch 50, DDIM Loss: 0.0269
Epoch 60, VAE_Loss: 1.687
Epoch 60, DDIM Loss: 0.02737
Epoch 70, VAE_Loss: 1.659
Epoch 70, DDIM Loss: 0.02704
Epoch 80, VAE_Loss: 1.606
Epoch 80, DDIM Loss: 0.02689
Epoch 90, VAE_Loss: 1.558
Epoch 90, DDIM Loss: 0.02688
Epoch 100, VAE_Loss: 1.576
Epoch 100, DDIM Loss: 0.02681
Epoch 110, VAE_Loss: 1.499
Epoch 110, DDIM Loss: 0.0269
Epoch 120, VAE_Loss: 1.453
Epoch 120, DDIM Loss: 0.02706
Epoch 130, VAE_Loss: 2.845
Epoch 130, DDIM Loss: 0.0271
Epoch 140, VAE_Loss: 1.825
Epoch 140, DDIM Loss: 0.02675
Epoch 150, VAE_Loss: 1.477
Epoch 150, DDIM Loss: 0.02657
Epoch 160, VAE_Loss: 1.274
Epoch 160, DDIM Loss: 0.02674
Epoch 170, VAE_Loss: 1.439
Epoch 170, DDIM Loss: 0.0

### Saving Models

In [33]:
torch.save(vae.state_dict(), "vae0.pt")
torch.save(ddim.state_dict(), "ddim0.pt")

### Loading saved models

In [26]:
loaded_vae = VAE(input_dim, latent_dim)
loaded_ddim = DDIM(latent_dim, timesteps)

loaded_vae.load_state_dict(torch.load('vae0.pt'))
loaded_ddim.load_state_dict(torch.load('ddim0.pt'))

loaded_vae.eval()
loaded_ddim.eval()

DDIM(
  (noise_predictor): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
  )
)

### reconstructing mel to audio

In [30]:
n_mels = 128
S_dB_min = 0  
S_dB_max = 1  
sr = 22050  
def denormalize_mel_spectrogram(mel_spec_tensor, S_dB_min, S_dB_max):
    mel_spec = mel_spec_tensor.numpy()  
    S_dB = mel_spec * (S_dB_max - S_dB_min) + S_dB_min
    return S_dB

def mel_spectrogram_to_audio(mel_spec_tensor, sr, S_dB_min, S_dB_max, n_fft=2048, hop_length=512, n_mels=128):
    
    S_dB = denormalize_mel_spectrogram(mel_spec_tensor, S_dB_min, S_dB_max)
    
    mel_basis = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)
    inv_mel_basis = np.linalg.pinv(mel_basis)
    
    if S_dB.shape[0] != n_mels:
        raise ValueError(f"Expected mel spectrogram with {n_mels} mel bins, but got {S_dB.shape[0]}")

    linear_spec = np.dot(inv_mel_basis, librosa.db_to_power(S_dB))
    
    audio = librosa.griffinlim(linear_spec, hop_length=hop_length)
    return audio




with torch.no_grad():
    for batch in val_loader:
        batch = batch[0]
        batch = batch.view(-1, input_dim)
        
        mu, logvar = loaded_vae.encoder(batch)
        z = loaded_vae.reparameterize(mu, logvar)
        
        estimated_z = loaded_ddim(z)
        
        reconstructed_batch = loaded_vae.decoder(estimated_z)
        
        reconstructed_audio = []
        for i, spec in enumerate(reconstructed_batch):
            spec = spec.view(n_mels, -1) 
            audio = mel_spectrogram_to_audio(spec, sr, S_dB_min, S_dB_max)
            reconstructed_audio.append(audio)

            sf.write(f'reconstructed_audio_{i}.wav', audio, sr)


# Evaluation 

In [ ]:
import torch
import torchaudio
from torch.utils.data import DataLoader

def extract_features(model, data_loader, device):
    model.to(device).eval()
    features = []
    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            feature = model(data)
            features.append(feature.cpu().numpy())
    features = np.vstack(features)
    return features


In [ ]:
def compute_statistics(features):
    mean = np.mean(features, axis=0)
    cov = np.cov(features, rowvar=False)
    return mean, cov


In [ ]:
from scipy.linalg import sqrtm

def frechet_distance(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return np.dot(diff, diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * np.trace(covmean)


In [ ]:
# Load your feature extraction model
model = load_pretrained_audio_model()

# Extract features
real_features = extract_features(model, real_data_loader, device='cuda')
generated_features = extract_features(model, generated_data_loader, device='cuda')

# Compute statistics
real_mean, real_cov = compute_statistics(real_features)
gen_mean, gen_cov = compute_statistics(generated_features)

# Compute Fréchet Distance
fad_score = frechet_distance(real_mean, real_cov, gen_mean, gen_cov)
print(f"FAD score: {fad_score}")
